In [3]:
pip install kaggle

  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105786 sha256=f846950199d4a7d47373130cb2090fabc197f81c7ac2e7efd14f28a390241bf6
  Stored in directory: /Users/yevheniiostapchenko/Library/Caches/pip/wheels/2b/af/a9/70bffa2773af622d2ebea9c8d407720b86e67bd40c465bf837
Successfully built kaggle
Note: you may need to restart the kernel to use updated packages.


In [1]:
!kaggle datasets download -d phucthaiv02/butterfly-image-classification

Dataset URL: https://www.kaggle.com/datasets/phucthaiv02/butterfly-image-classification
License(s): CC0-1.0
100%|███████████████████████████████████████▊| 225M/226M [00:07<00:00, 39.0MB/s]
100%|████████████████████████████████████████| 226M/226M [00:07<00:00, 30.2MB/s]


In [3]:
mkdir data

In [5]:
!unzip butterfly-image-classification.zip -d data

Archive:  butterfly-image-classification.zip
  inflating: data/Testing_set.csv    
  inflating: data/Training_set.csv   
  inflating: data/test/Image_1.jpg   
  inflating: data/test/Image_10.jpg  
  inflating: data/test/Image_100.jpg  
  inflating: data/test/Image_1000.jpg  
  inflating: data/test/Image_1001.jpg  
  inflating: data/test/Image_1002.jpg  
  inflating: data/test/Image_1003.jpg  
  inflating: data/test/Image_1004.jpg  
  inflating: data/test/Image_1005.jpg  
  inflating: data/test/Image_1006.jpg  
  inflating: data/test/Image_1007.jpg  
  inflating: data/test/Image_1008.jpg  
  inflating: data/test/Image_1009.jpg  
  inflating: data/test/Image_101.jpg  
  inflating: data/test/Image_1010.jpg  
  inflating: data/test/Image_1011.jpg  
  inflating: data/test/Image_1012.jpg  
  inflating: data/test/Image_1013.jpg  
  inflating: data/test/Image_1014.jpg  
  inflating: data/test/Image_1015.jpg  
  inflating: data/test/Image_1016.jpg  
  inflating: data/test/Image_1017.jpg  
  inf

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
pip install pandas tensorflow

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [9]:
train_csv_path = "data/Training_set.csv"  
test_csv_path  = "data/Testing_set.csv"

train_df = pd.read_csv(train_csv_path)
test_df  = pd.read_csv(test_csv_path)

In [10]:
# Dla danych treningowych
train_df['filename'] = train_df['filename'].apply(lambda x: os.path.join("data/train", x))

# Dla danych testowych
test_df['filename'] = test_df['filename'].apply(lambda x: os.path.join("data/test", x))

In [11]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 20% na walidację
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

Found 5200 validated image filenames belonging to 75 classes.
Found 1299 validated image filenames belonging to 75 classes.


In [12]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filename',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

Found 2786 validated image filenames.


In [15]:
num_classes = len(train_generator.class_indices)

In [16]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

num_classes = len(train_generator.class_indices)
print("Detected classes:", train_generator.class_indices)
print("Number of classes:", num_classes)

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

Found 5200 validated image filenames belonging to 75 classes.
Found 1299 validated image filenames belonging to 75 classes.
Detected classes: {'ADONIS': 0, 'AFRICAN GIANT SWALLOWTAIL': 1, 'AMERICAN SNOOT': 2, 'AN 88': 3, 'APPOLLO': 4, 'ATALA': 5, 'BANDED ORANGE HELICONIAN': 6, 'BANDED PEACOCK': 7, 'BECKERS WHITE': 8, 'BLACK HAIRSTREAK': 9, 'BLUE MORPHO': 10, 'BLUE SPOTTED CROW': 11, 'BROWN SIPROETA': 12, 'CABBAGE WHITE': 13, 'CAIRNS BIRDWING': 14, 'CHECQUERED SKIPPER': 15, 'CHESTNUT': 16, 'CLEOPATRA': 17, 'CLODIUS PARNASSIAN': 18, 'CLOUDED SULPHUR': 19, 'COMMON BANDED AWL': 20, 'COMMON WOOD-NYMPH': 21, 'COPPER TAIL': 22, 'CRECENT': 23, 'CRIMSON PATCH': 24, 'DANAID EGGFLY': 25, 'EASTERN COMA': 26, 'EASTERN DAPPLE WHITE': 27, 'EASTERN PINE ELFIN': 28, 'ELBOWED PIERROT': 29, 'GOLD BANDED': 30, 'GREAT EGGFLY': 31, 'GREAT JAY': 32, 'GREEN CELLED CATTLEHEART': 33, 'GREY HAIRSTREAK': 34, 'INDRA SWALLOW': 35, 'IPHICLUS SISTER': 36, 'JULIA': 37, 'LARGE MARBLE': 38, 'MALACHITE': 39, 'MANGROVE SK

/Users/yevheniiostapchenko/anaconda3/envs/butterfly_env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/yevheniiostapchenko/anaconda3/envs/butterfly_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 4285s 26s/step - accuracy: 0.0493 - loss: 4.4317 - val_accuracy: 0.2179 - val_loss: 3.1073
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 3850s 24s/step - accuracy: 0.4330 - loss: 2.2895 - val_accuracy: 0.3703 - val_loss: 2.4912
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 3743s 23s/step - accuracy: 0.8150 - loss: 0.7267 - val_accuracy: 0.3826 - val_loss: 2.9970
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 3824s 23s/step - accuracy: 0.9654 - loss: 0.1662 - val_accuracy: 0.3918 - val_loss: 3.1756
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 3683s 23s/step - accuracy: 0.9934 - loss: 0.0413 - val_accuracy: 0.3949 - val_loss: 3.7872
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 3645s 22s/step - accuracy: 0.9991 - loss: 0.0103 - val_accuracy: 0.3865 - val_loss: 4.0120
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 3699s 23s/step - accuracy: 0.9941 - loss: 0.0248 - val_accuracy: 0.3626 - val_loss: 4.3415
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 3771s 23s/step - accuracy: 0.9879 - loss: 0

In [17]:
model.save('butterfly_classifier.h5')

In [18]:
pip install coremltools

Note: you may need to restart the kernel to use updated packages.


In [27]:
!pip install --upgrade "tensorflow-macos==2.12.*"
!pip install --upgrade "coremltools"
!pip install --upgrade "scikit-learn==1.5.1"

  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.8/200.8 MB 8.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 45.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 38.7 MB/s eta 0:00:0000:0100:01
Using cached cachetools-5.5.0-py3-none-any.whl (9.5 kB)
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3
    Uninstalling protobuf-5.29.3:
      Successfully uninstalled protobuf-5.29.3
  Attempting uninstall: numpy
    Found existing installation: nu

In [30]:
model.export("my_tf_model")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unknown node type <gast.gast.Expr object at 0x17f4932e0>
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unknown node type <gast.gast.Expr object at 0x17f487070>
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Saved artifact at 'my_tf_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 75), dtype=tf.float32, name=None)
Captures:
  6287287744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6295477408: Tenso

In [33]:
import tensorflow as tf
import coremltools as ct

# Wczytaj folder "my_tf_model" (legacy SavedModel)
loaded_model = tf.saved_model.load("my_tf_model")
# Zobacz, jakie ma sygnatury:
print("Signatures:", loaded_model.signatures)

Signatures: _SignatureMap({'serve': <ConcreteFunction (*, keras_tensor: TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')) -> Dict[['output_0', TensorSpec(shape=(None, 75), dtype=tf.float32, name='output_0')]] at 0x17AAB8040>, 'serving_default': <ConcreteFunction (*, keras_tensor: TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')) -> Dict[['output_0', TensorSpec(shape=(None, 75), dtype=tf.float32, name='output_0')]] at 0x17B6ADEE0>})


In [34]:
concrete_func = loaded_model.signatures["serving_default"]

In [37]:
model_keras.save("my_model.keras")

In [43]:
model_keras = tf.keras.models.load_model("butterfly_classifier.h5")

In [47]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 40.8 MB/s eta 0:00:00a 0:00:01


In [49]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Keras version (within TF):", tf.keras.__version__)

TensorFlow version: 2.18.0
Keras version (within TF): 3.8.0


In [50]:
import tensorflow as tf
model_keras = tf.keras.models.load_model("butterfly_classifier.h5")

In [51]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

# Zakładamy, że obraz wejściowy ma kształt [None, 224, 224, 3].
# Dopasuj go do swojego modelu jeśli jest inny.
concrete_func = tf.function(model_keras).get_concrete_function(
    tf.TensorSpec([None, 224, 224, 3], tf.float32)
)

Cause: Unable to locate the source code of <function Layer.__call__ at 0x169a35670>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [52]:
frozen_func = convert_variables_to_constants_v2(
    concrete_func, lower_control_flow=False, aggressive_inlining=True
)
graph_def = frozen_func.graph.as_graph_def()

I0000 00:00:1737252936.508172 3612074 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1737252936.513282 3612074 single_machine.cc:361] Starting new session


In [54]:
import coremltools as ct

mlmodel = ct.convert(
    graph_def,
    source="tensorflow",
    # Możesz jawnie ustawić parametry wejścia/wyjścia:
    inputs=[ct.ImageType(shape=(1,224,224,3))],
    # 'outputs' = nazwa węzła wyjściowego (np. 'Identity'), jeśli musisz
)
mlmodel.save("ButterflyClassifier.mlpackage")

Converting TF Frontend ==> MIL Ops: 100%|███| 27/27 [00:00<00:00, 2891.44 ops/s]
Running MIL frontend_tensorflow2 pipeline: 100%|█| 7/7 [00:00<00:00, 1749.19 pas
Running MIL default pipeline: 100%|████████| 89/89 [00:01<00:00, 77.56 passes/s]
Running MIL backend_mlprogram pipeline: 100%|█| 12/12 [00:00<00:00, 140.50 passe


In [55]:
!zip -r ButterflyClassifier.zip ButterflyClassifier.mlpackage

  adding: ButterflyClassifier.mlpackage/ (stored 0%)
  adding: ButterflyClassifier.mlpackage/Manifest.json (deflated 59%)
  adding: ButterflyClassifier.mlpackage/.ipynb_checkpoints/ (stored 0%)
  adding: ButterflyClassifier.mlpackage/.ipynb_checkpoints/Manifest-checkpoint.json (deflated 59%)
  adding: ButterflyClassifier.mlpackage/Data/ (stored 0%)
  adding: ButterflyClassifier.mlpackage/Data/com.apple.CoreML/ (stored 0%)
  adding: ButterflyClassifier.mlpackage/Data/com.apple.CoreML/weights/ (stored 0%)
 (deflated 9%)erflyClassifier.mlpackage/Data/com.apple.CoreML/weights/weight.bin
  adding: ButterflyClassifier.mlpackage/Data/com.apple.CoreML/model.mlmodel (deflated 79%)
